<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2022/ex10noteA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex10noteA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2022)


----
## 準備
----

Google Colab の Notebook では， Python というプログラミング言語のコードを動かして計算したりグラフを描いたりできます．
Python は，機械学習・人工知能やデータサイエンスの分野ではメジャーなプログラミング言語ですが，それを学ぶことはこの授業の守備範囲ではありません．以下の所々に現れるプログラムっぽい記述の内容は，理解できなくて構いません．

以下，コードセルを上から順に実行してながら読んでいってね．

In [ ]:
# 準備あれこれ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
seaborn.set()

----
## 過適合の抑制とモデル選択(1)
----

「汎化と過適合」の回に，**過適合**(over-fitting)の話をしました．今回は，過適合を抑制して汎化能力の高い学習モデルを得る方法について説明します．



---
### 例題

説明のための例題として，多項式を当てはめる回帰の問題を考えます．

In [ ]:
## データを生成
#
def gendat(N, true_data=False, seed=0, sigma=0.0):

    if true_data:
        x = np.linspace(-0.1, 1.1, num=N)
        y = np.sin(2*np.pi*x)
    else:
        np.random.seed(seed)
        x = np.linspace(0.0, 1.0, num=N)
        y = np.sin(2*np.pi*x) + sigma*np.random.randn(N)

    return np.vstack((x, y)).T

## 1, x, x^2, x^3, ..., X^D をならべたデータ行列（N x (D+1)）をつくる
#
def makeDataMatrix(x, D):

    N = x.shape[0]
    X = np.zeros((N, D+1))
    X[:, 0] = 1
    for i in range(1, D+1):
        X[:, i] = x**i

    return X

##  正則化ありの最小二乗法
#
def solve2(X, y, alpha=0.0):

    A = np.dot(X.T, X)
    b = np.dot(X.T, y)
    if alpha > 0.0:
        Dp1 = X.shape[1]
        Id = np.eye(Dp1)
        Id[0, 0] = 0.0  # バイアス項のみ正則化の対象外とする
        A += alpha * Id

    # x is the solution of the equation Ax = b
    x = np.linalg.solve(A, b)

    return x

次のセルを実行すると，データをグラフに描きます．

In [ ]:
ndata = 20   # 学習データ・テストデータの数
sigma = 0.1  # それらに乗せるノイズの大きさ

# ノイズの乗った学習データ
datL = gendat(ndata, seed=0, sigma=sigma)

# 同じく検証データ
datV = gendat(ndata, seed=1, sigma=sigma)

# 真の値
datTrue = gendat(1000, true_data=True)

# グラフを描く
fig, ax = plt.subplots(facecolor="white", figsize=(8, 6))
ax.set_xlim(-0.1, 1.1)
ax.set_ylim(-1.2, 1.2)
ax.plot(datTrue[:, 0], datTrue[:, 1], color="gray", label="true function")
ax.scatter(datL[:, 0], datL[:, 1], color="red", label="learning data")
ax.scatter(datV[:, 0], datV[:, 1], color="blue", label="validation data")
ax.legend()
plt.show()

この問題は，赤い点で描かれた学習データに多項式を当てはめるというものです．これらの学習データは，$[0, 1]$ の範囲の $x$ の値に対して，$y = \sin{2\pi x} + \textrm{ノイズ}$ として作られています．観測されたデータに小さなノイズが乗っているという状況です．ノイズの値は，平均 $0$ 標準偏差 $0.1$ の正規分布をする乱数としています．

青い点は，学習データとは別に作ったデータです．作り方は全く同じですが，ノイズの値が異なるために異なる値をとっています．
赤い点の学習データを用いて学習（最小二乗法による多項式あてはめ）を行い，青い点のデータに対する誤差を測ることで，得られたモデルの汎化性能を検証します．

この青い点のデータのように，学習モデルの汎化性能を検証・評価するために，学習データとは別に用意するデータのことを，**検証データ**(validation data)と呼びます．

次のセルを実行すると，最小二乗法によって学習データに$D$次多項式を当てはめた結果を表示します．また，学習データと検証データに対する誤差（平均二乗誤差）の値も表示します．



In [ ]:
#@title 多項式の次数 D をいろいろ変えてこのセルを何度か実行し直してみよう {run: "auto"}
D = 1 #@param{type:"slider", min:1, max:20, step:1}

# 学習データにD次多項式を当てはめて平均二乗誤差を計算
XL = makeDataMatrix(datL[:, 0], D)
yL = datL[:, 1]
w = solve2(XL, yL)
yL_est = XL @ w
msqeL = np.mean((yL - yL_est)**2)

# 検証データに対する平均二乗誤差を計算
XV = makeDataMatrix(datV[:, 0], D)
yV = datV[:, 1]
yV_est = XV @ w
msqeV = np.mean((yV - yV_est)**2)

# 真の値に対する予測値を計算
datTrue = gendat(200, true_data=True)
X = makeDataMatrix(datTrue[:, 0], D)
y = datTrue[:, 1]
y_est = X @ w

# グラフを描く
fig, ax = plt.subplots(facecolor="white", figsize=(8, 6))
ax.set_xlim(-0.1, 1.1)
ax.set_ylim(-1.2, 1.2)
ax.plot(datTrue[:, 0], datTrue[:, 1], color="gray", label="true function")
ax.scatter(datL[:, 0], datL[:, 1], color="red", label="learning data")
ax.scatter(datV[:, 0], datV[:, 1], color="blue", label="validation data")
ax.plot(datTrue[:, 0], y_est, color="green", label="estimated curve")
ax.legend()
plt.show()

print(f'### D = {D}')
print('# 得られたパラメータ（多項式の係数） = ', w)
print(f'# 学習データに対する平均二乗誤差 = {msqeL:.3e}')
print(f'# 検証データに対する平均二乗誤差 = {msqeV:.3e}')

学習データまたは検証データを $\{(x_n, y_n)\}, n = 1, 2, \ldots, N$ として，学習モデルの入出力を $f(x)$ で表すとき，平均二乗誤差は次式の通りです．
$$
\frac{1}{N}\sum_{n=1}^{N}(y_n - f(x_n))^2
$$
最小二乗法の定式化で出てきた二乗誤差の和とは定数倍違っているだけ（$1/2$だったのが$1/N$になっただけ）ですので，意味合いは同じです．

上記のセルを実行すると，$D$を大きくするとモデルの予測値が赤い点に近づいていき，学習データに対する平均二乗誤差も小さくなっていることが分かります．しかし，青い点からは離れてしまうところもあって，検証データに対する平均二乗誤差はむしろ大きくなっていることがわかります．

上記の観察結果をもう少しきちんと確認するために，$D$を $1$ から $20$ まで変化させたときの学習データと検証データそれぞれに対する平均二乗誤差を求めてグラフに描いてみましょう．

In [ ]:
## D を 1 から 20 まで変えて平均二乗誤差を求めてみる

Dlist = np.arange(1, 21, dtype=int)
msqeLlist = np.empty(len(Dlist))
msqeVlist = np.empty(len(Dlist))

for id, D in enumerate(Dlist):

    # 学習データにD次多項式を当てはめて平均二乗誤差を計算
    XL = makeDataMatrix(datL[:, 0], D)
    yL = datL[:, 1]
    w = solve2(XL, yL)
    yL_est = XL @ w
    msqeLlist[id] = np.mean((yL - yL_est)**2)

    # テストデータに対する平均二乗誤差を計算
    XV = makeDataMatrix(datV[:, 0], D)
    yV = datV[:, 1]
    yV_est = XV @ w
    msqeVlist[id] = np.mean((yV - yV_est)**2)

# グラフを描く
fig, ax = plt.subplots(facecolor="white", figsize=(8, 6))
ax.set_xlim(0, 21)
ax.set_ylim(0, 0.04)
ax.set_xticks(np.arange(1, 21))
ax.set_xlabel('D')
ax.set_ylabel('mean squared error')
ax.scatter(Dlist, msqeLlist, color="red", label="msqeL")
ax.scatter(Dlist, msqeVlist, color="blue", label="msqeV")
ax.legend()
plt.show()

ii = np.argmin(msqeVlist)
print(f'msqeV の最小値は {msqeVlist[ii]:.3f} (D = {Dlist[ii]})')

この結果から，大まかな傾向として，次のようなことが読み取れます．
- $D$が小さすぎると，学習データに対しても検証データに対しても誤差が大きい．
上の図では，$D=1,2$のときの誤差の値は縦軸の上限より大きくて表示されていません．
- $D$が大きくなるにつれて学習データに対する誤差は減少する傾向にあるが，検証データに対する誤差はいったん減ってから再び増加する傾向にある．

この例題については，汎化性能の観点からは，$D$が小さすぎても大きすぎてもよくない，ということのようです．

---
### パラメータの正則化


以前にも説明した通り，パラメータの数が多い学習モデルは，自由度が大きいためうまく学習データに当てはまるのだけれど，学習データに当てはまりすぎて過適合を起こしやすくなります（下図参照）．
上記の実験の結果も，そのことを示しています．

<img width="50%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/modeldof.png">

一般に，汎化性能を高めるためには，学習モデルの自由度を適切に定めることが望ましいです．
しかし，どれくらいが適切かは問題の複雑さや学習モデルの性質によるものであり，あらかじめ知ることは困難です．
そのため，モデルの構造を固定したままでパラメータの実質的な自由度を調節するという手法が考えられています．
以下で説明する「正則化」は，そのような手法の代表例です．




#### 正則化とは

上記の実験の結果をよく観察すると，パラメータ数すなわち多項式の次数$D$が大きい場合，学習によって得られるパラメータの値（の絶対値）がずいぶん大きくなっていることが分かります．そのために，モデルが表す曲線の形が過剰に複雑になって，過適合を起こしています．
もしも，次数が大きくてもパラメータの値があまり大きくならないように抑えることができれば，曲線の形が複雑になりすぎず，過適合を抑制できるかもしれません．

そこで，パラメータの値が大きくなりすぎるのを防ぐために，パラメータの二乗和を学習の目的関数に付け足したものを新たな目的関数として，その値を最小化する，という学習の方法が考えられています．これを，パラメータの **正則化** (regularization)といいます．



以下，さきほどの多項式当てはめの問題の例で正則化の方法を説明します（注）．
この場合，学習モデルは
$$
f(x) = w_0 + w_1x + w_2x^2 + \cdots + w_Dx^D
$$
という式で表され，$w_0, w_1, \ldots, w_D$ がそのパラメータです．
学習データを $\{(x_n, y_n)\}, n = 1, 2, \ldots, N$ とすると，通常の最小二乗法による多項式当てはめでは，以下の式で表される $E$ を目的関数として，この値を最小にするパラメータを求めていました．
$$
E = \frac{1}{2}\sum_{n=1}^{N}(y_n - f(x_n))^2
$$

※注: 正則化は，多項式当てはめに限らず，機械学習の様々な問題に適用できる手法です．

これに対して，正則化を行う場合，次のような目的関数を最小化します．
$$
F = E + \frac{\alpha}{2}\sum_{d=1}^{D}w_d^2
$$
この目的関数 $F$ は，$E$に，モデルのパラメータの二乗和の項（**正則化項**）を付け足したものとなっています（注）．
通常の最小二乗法では$E$を最小化しますが，かわりに$F$を最小化することで，学習データに対する二乗誤差を小さくするだけでなく，パラメータの二乗和も同時に小さくすることができます．
ここで，$\alpha$は正の定数であり，学習データに対する二乗誤差の項と正則化項のバランスを決めるハイパーパラメータです．

※この正則化項では，バイアス項 $w_0$ を二乗和から除外しています．ここで考えている学習モデルの場合，バイアス項には正則化の影響を及ぼさない方がより適切だからです．

#### 例題に正則化ありの最小二乗法を適用してみる

次のセルでは，多項式当てはめの例題で $D=20$とパラメータ数を多く設定した場合について，正則化付きの最小二乗法を適用した結果を示します．比較のため，正則化なしの結果も示しています．

In [ ]:
D = 20
alpha = 0.0002  # 正則化項の重み

# 学習データにD次多項式を当てはめて平均二乗誤差を計算
XL = makeDataMatrix(datL[:, 0], D)
yL = datL[:, 1]
w0 = solve2(XL, yL)              # 正則化なし
wa = solve2(XL, yL, alpha=alpha) # 正則化あり
yL_est0 = XL @ w0
yL_esta = XL @ wa
msqeL0 = np.mean((yL - yL_est0)**2)
msqeLa = np.mean((yL - yL_esta)**2)

# 検証データに対する平均二乗誤差を計算
XV = makeDataMatrix(datV[:, 0], D)
yV = datV[:, 1]
yV_est0 = XV @ w0
yV_esta = XV @ wa
msqeV0 = np.mean((yV - yV_est0)**2)
msqeVa = np.mean((yV - yV_esta)**2)

# 真の値に対する予測値を計算
datTrue = gendat(200, true_data=True)
X = makeDataMatrix(datTrue[:, 0], D)
y = datTrue[:, 1]
y_est0 = X @ w0
y_esta = X @ wa

# グラフを描く
fig, ax = plt.subplots(facecolor="white", figsize=(8, 6))
ax.set_xlim(-0.1, 1.1)
ax.set_ylim(-1.2, 1.2)
ax.plot(datTrue[:, 0], datTrue[:, 1], color="gray", label="true function")
ax.scatter(datL[:, 0], datL[:, 1], color="red", label="learning data")
ax.scatter(datV[:, 0], datV[:, 1], color="blue", label="validation data")
ax.plot(datTrue[:, 0], y_est0, color="green", label=r"$\alpha=0$", linestyle="dashed")
ax.plot(datTrue[:, 0], y_esta, color="green", label=f"$\\alpha={alpha}$")
ax.legend()
plt.show()

print(f'### D = {D}, alpha = 0')
print('# 得られたパラメータ（多項式の係数） = ', w0)
print(f'# 学習データに対する平均二乗誤差 = {msqeL0:.3e}')
print(f'# 検証データに対する平均二乗誤差 = {msqeV0:.3e}')
print()
print(f'### D = {D}, alpha = {alpha}')
print('# 得られたパラメータ（多項式の係数） = ', wa)
print(f'# 学習データに対する平均二乗誤差 = {msqeLa:.3e}')
print(f'# 検証データに対する平均二乗誤差 = {msqeVa:.3e}')

得られたパラメータの値を見ると，正則化のおかげでパラメータの値の絶対値が小さくなっていることがわかります．
そのことはグラフの方にも反映されており，正則化なしの場合（緑の破線）に比べて，正則化ありの方（緑の実線）が曲線の急激な変化が抑えられてることが見てとれます．
検証データに対する平均二乗誤差の値もより小さくなっており，正則化によって汎化性能を改善できていることが確認できます．


上記では，$\alpha = 0.0002$ としたときの結果を見ましたが，$\alpha$を様々に変化させるとどうなるか，平均二乗誤差のグラフを描いて考えてみましょう．

In [ ]:
D = 20

alphalist = np.arange(0, 0.001, 0.00001)
msqeLlist = np.empty(len(alphalist))
msqeVlist = np.empty(len(alphalist))

for i, alpha in enumerate(alphalist):

    # 学習データにD次多項式を当てはめて平均二乗誤差を計算
    XL = makeDataMatrix(datL[:, 0], D)
    yL = datL[:, 1]
    w = solve2(XL, yL, alpha)
    yL_est = XL @ w
    msqeLlist[i] = np.mean((yL - yL_est)**2)

    # テストデータに対する平均二乗誤差を計算
    XV = makeDataMatrix(datV[:, 0], D)
    yV = datV[:, 1]
    yV_est = XV @ w
    msqeVlist[i] = np.mean((yV - yV_est)**2)

    #print(f'D = {D}, msqeL = {msqeLlist[id]:.3e}, msqeT = {msqeVlist[id]:.3e}')


# グラフを描く
fig, ax = plt.subplots(facecolor="white", figsize=(8, 6))
ax.set_ylim(0, 0.03)
ax.set_xlabel('alpha')
ax.set_ylabel('mean squared error')
ax.scatter(alphalist, msqeLlist, color="red", s=5, label="msqeL")
ax.scatter(alphalist, msqeVlist, color="blue", s=5, label="msqeV")
ax.legend()
plt.show()

ii = np.argmin(msqeVlist)
print(f'正則化なし（alpha = 0 のとき）の msqeV = {msqeVlist[0]:.3f}')
print(f'msqeV の最小値は {msqeVlist[ii]:.3f} (alpha = {alphalist[ii]})')

$F$の式を見ると，$\alpha = 0$ の場合，通常の最小二乗法の目的関数 $E$ に一致することがわかります．
上記のグラフの左端の点がそのときの平均二乗誤差の値です．

そこから $\alpha$を大きくしていくと，学習データに対する誤差の項よりも正則化項の効き目が強くなります．
すると，学習データに対する誤差が増加する一方で，検証データに対する誤差が減少していきます．
しかし，$\alpha$を大きくしすぎると，今度は正則化項の効き目が強すぎて，どちらの誤差も大きくなってしまっています．

正則化項の重み $\alpha$ は手動で調節するハイパーパラメータですが，このように，適切な値を選択してやる必要があります．その方法については，「モデル選択」のセクションで説明します．



#### 【発展】$L_2$正則化と$L_1$正則化

上記の正則化の説明では，パラメータの二乗和を最小化していました．パラメータをならべたベクトルを
$$
\mathbf{w} = (w_1, w_2, \ldots, w_D)
$$
のように表すと，パラメータの二乗和は，ベクトル$\mathbf{w}$のユークリッドノルム（$L_2$ノルム）の二乗です．
$$
||\mathbf{w}||_2^2 = \sum_{d=1}^{D}w_d^2
$$
そのため，ここで説明している正則化の方法は，「$L_2$正則化」と呼ばれることもあります．

正則化の方法としてはこの「$L_2$正則化」が一般的ですが，場合によっては，$L_1$ノルム
$$
||\mathbf{w}||_1^2 = \sum_{d=1}^{D}|w_d|
$$
を用いる「$L_1$正則化」が用いられることもあります．ここではこれ以上説明しませんが，「$L_2$正則化」とはまた違った効果を発揮します．

$L_2$正則化を用いる回帰は「リッジ回帰」（ridge regression），$L_1$正則化を用いる回帰は「ラッソ回帰」（LASSO(Least Absolute Shrinkage and Selection Operator) regression）と呼ばれることもあります．



#### 【発展】正則化付きの最小二乗法の正規方程式と，「正則化」の意味

「回帰のための教師あり学習(2) 平面の当てはめ」の notebook で，平面当てはめの最小二乗法における正規方程式を導出しました．
ここでは，正則化項を付加するとその形がどのように変わるかを示します．


正則化項なし場合，平面当てはめの最小二乗法の正規方程式は，以下の式(4)の形となるのでした．

> 個々の学習データ（の先頭に1を付け足したもの）$\mathbf{x}_n$ を $(D+1)\times 1$ 行列（列ベクトル）とみなし，これをならべた行列を $X$ と表記します．つまり $X$は
$$
X = (\mathbf{x}_1\ \mathbf{x}_2\ \cdots\ \mathbf{x}_N)
$$
という $(D+1)\times N$ の行列です．また，正解の値 $y_n$ をならべた $1 \times N$ 行列を $Y$ と表記します．
$$
Y = (y_1\ y_2\ \ldots\ y_N)
$$
このとき，正規方程式は次式のようになります（正規方程式がこのようになることを示すのは後の節に回します）．
$$
XX^{\top}\mathbf{w} = XY^{\top} \qquad (4)
$$
ただし，$\mathbf{w}$ はパラメータをならべた $(D+1)\times 1$ 行列です．
この式の形からわかるように，正規方程式は $(D+1)$元の連立方程式です．


これに対して，正則化項付きの場合，正規方程式は次の形になります（大学初年次レベルの線形代数と微積分の知識があれば証明できますが，ここでは省略します）．
$$
(XX^{\top} + \alpha I)\mathbf{w} = XY^{\top} \qquad (*)
$$

式(4)の正規方程式の左辺の行列 $X$ の大きさは $(D+1)\times N$ ，$XX^{\top}$ の大きさは $(D+1)\times (D+1)$ です．
データ数$N$がパラメータ数$(D+1)$に比べて少なかったり，多かったとしても同じ値の繰り返しだったりすると，行列$XX^{\top}$のランクが $D+1$ より小さくなり得ます．その場合，$XX^{\top}$ は非正則となり逆行列が存在しないので，正規方程式の解がまともに求まらなくなります．

ところが，$\textrm{rank}(XX^{\top}) < D+1$ となっているような場合でも，$XX^{\top} + \alpha I$ の固有値は $\alpha$ 以上となるため，$XX^{\top} + \alpha I$ はフルランク，正則になります（注）．したがって，式(*)の正規方程式がちゃんと唯一の解を持ちます．

このような話がベースにあるため，ここで説明した方法は「正則化」と呼ばれています．

※注: このことは，任意の正方行列$A$の固有値と固有ベクトルを$\lambda$と$\mathbf{u}$としたとき，$B = A + \alpha I$ に対して $B\mathbf{u} = A\mathbf{u} + \alpha\mathbf{u} = (\lambda+\alpha)\mathbf{u}$ が成り立つこと，すなわち，$B$の固有値が $\lambda+\alpha$となることから示せます．